Importações de bibliotecas para manipulação de dados (pandas), arquivos (os, glob) e expressões regulares (re).

Desativa avisos de atribuições encadeadas no pandas


In [ ]:
import pandas as pd
import os
import glob
import re

pd.options.mode.chained_assignment = None

Carrega os dados do arquivo CSV contendo códigos e nomes de cidades em um DataFrame

Converte o DataFrame em um dicionário

Cria um dicionário de conversão onde as chaves são os IDs e os valores são os nomes dos municípios

Retorna o dicionário de conversão


In [ ]:
# Carrega os dados do arquivo CSV contendo códigos e nomes de cidades em um DataFrame
# Converte o DataFrame em um dicionário
# Cria um dicionário de conversão onde as chaves são os IDs e os valores são os nomes dos municípios
# Retorna o dicionário de conversão

conv = pd.read_csv("Dados/Conv_codigo_nome_cidade.csv")
conv_dict = conv.to_dict()
conversor = {conv_dict['ID'][f]:conv_dict['Município'][f] for f in conv_dict['ID']}
conversor


Obtém a lista de caminhos de arquivos CSV de receitas na pasta 'Dados'.

Inicializa um dicionário para armazenar os DataFrames de receitas.

Para cada caminho na lista de caminhos:

- Extrai o ano do nome do arquivo usando expressões regulares.
- Carrega o arquivo CSV em um DataFrame, pulando as primeiras 2 linhas, definindo a segunda linha como cabeçalho.
- Filtra o DataFrame para manter apenas as colunas específicas: "Cod.IBGE", "População", "Coluna", "Conta", "Valor".
- Adiciona o DataFrame ao dicionário, usando o ano como chave.

In [ ]:
path_planilhas_receitas = glob.glob("Dados/Receita *.csv")
planilhas_receitas = {}
for path in path_planilhas_receitas:
    planilhas_receitas.update({ re.findall(r"\d{4}", path)[0]: 
        pd.read_csv(path, encoding="ANSI", skiprows=2, header=1,sep=";", decimal=",").filter(items=["Cod.IBGE", "População", "Coluna", "Conta", "Valor"])
        })

---

Inicializa um DataFrame vazio para armazenar os dados finais.

Para cada ano nas planilhas de receitas:

- Seleciona as receitas brutas realizadas.
- Ajusta os códigos das contas dependendo do ano (2022 tem códigos diferentes).
- Para o imposto sobre o patrimônio:
  - Filtra as linhas correspondentes à conta de imposto sobre o patrimônio.
  - Substitui os códigos IBGE pelos nomes dos municípios usando o dicionário de conversão.
  - Remove as colunas "Coluna" e "Conta".
  - Renomeia a coluna "Valor" para "Impostos sobre o Patrimônio".
  - Adiciona uma coluna "Ano" com o respectivo ano.
  - Define o índice como "Cod.IBGE" e "Ano".

- Para o imposto sobre serviços:
  - Filtra as linhas correspondentes à conta de imposto sobre serviços.
  - Substitui os códigos IBGE pelos nomes dos municípios usando o dicionário de conversão.
  - Remove as colunas "Coluna" e "Conta".
  - Renomeia a coluna "Valor" para "Impostos sobre Serviços".
  - Adiciona uma coluna "Ano" com o respectivo ano.
  - Define o índice como "Cod.IBGE" e "Ano".

- Para o imposto sobre renda retido na fonte:
  - Filtra as linhas correspondentes à conta de imposto sobre renda.
  - Substitui os códigos IBGE pelos nomes dos municípios usando o dicionário de conversão.
  - Remove as colunas "Coluna" e "Conta".
  - Renomeia a coluna "Valor" para "Impostos sobre Renda Retido na fonte".
  - Adiciona uma coluna "Ano" com o respectivo ano.
  - Define o índice como "Cod.IBGE" e "Ano".

- Mescla os DataFrames de imposto sobre o patrimônio e imposto sobre serviços usando o índice, mantendo todas as linhas.
- Mescla o resultado com o DataFrame de imposto sobre renda retido na fonte, mantendo todas as linhas.
- Remove colunas duplicadas geradas pela mesclagem.
- Se o DataFrame final estiver vazio, atribui o DataFrame completo.
- Caso contrário, concatena o DataFrame completo ao DataFrame final.


In [ ]:
df_final = pd.DataFrame()

for ano in planilhas_receitas:
    planilha = planilhas_receitas[ano]
    
    receitas = planilha[planilha['Coluna'] == "Receitas Brutas Realizadas"]
    
    if ano != "2022":
        # imposto_patrimonio = receitas[receitas['Conta'] == "1.1.1.8.01.0.0 Impostos sobre o Patrimônio para Estados/DF/Municípios"]
        imposto_patrimonio = receitas[receitas['Conta'].str.contains(r"(1.1.1.8.01.0.0)")]
        # imposto_iss = receitas[receitas['Conta'] == "1.1.1.8.02.3.0 Imposto sobre Serviços de Qualquer Natureza"]
        imposto_iss = receitas[receitas['Conta'].str.contains(r"(1.1.1.8.02.3.0)")]
    else:
        # No ano de 2022 algumas contas tiveram o codigo alterado, com isso é necessario o ajuste:
        imposto_patrimonio = receitas[receitas['Conta'].str.contains(r"(1.1.1.2.00.0.0)")]
        imposto_iss = receitas[receitas['Conta'].str.contains(r"(1.1.1.4.51.1.0)")]
    
    #imposto_rf = receitas[receitas['Conta'] == "1.1.1.3.00.0.0 - Impostos sobre a Renda e Proventos de Qualquer Natureza"]
    # O imposto de renda não preciso do ajuste
    imposto_rf = receitas[receitas['Conta'].str.contains(r"(1.1.1.3.00.0.0)")]
    
    imposto_patrimonio.loc[:, "Municipio"] = imposto_patrimonio.replace({"Cod.IBGE": conversor})['Cod.IBGE'].tolist()
    imposto_patrimonio.drop(["Coluna", "Conta"], axis=1,inplace=True)
    imposto_patrimonio = imposto_patrimonio.rename(columns={"Valor":"Impostos sobre o Patrimônio"})
    imposto_patrimonio.loc[:, "Ano"] = ano
    imposto_patrimonio.set_index(["Cod.IBGE", "Ano"], inplace=True)
    

    imposto_iss.loc[:, "Municipio"] = imposto_iss.replace({"Cod.IBGE": conversor})['Cod.IBGE'].tolist()
    imposto_iss.drop(["Coluna", "Conta"], axis=1,inplace=True)
    imposto_iss = imposto_iss.rename(columns={"Valor":"Impostos sobre Serviços"})
    imposto_iss.loc[:, "Ano"] = ano
    imposto_iss.set_index(["Cod.IBGE", "Ano"], inplace=True)
    

    imposto_rf.loc[:, "Municipio"] = imposto_rf.replace({"Cod.IBGE": conversor})['Cod.IBGE'].tolist()
    imposto_rf.drop(["Coluna", "Conta"], axis=1,inplace=True)
    imposto_rf = imposto_rf.rename(columns={"Valor":"Impostos sobre Renda Retido na fonte"})
    imposto_rf.loc[:, "Ano"] = ano
    imposto_rf.set_index(["Cod.IBGE", "Ano"], inplace=True)
    
    df_mesclada = imposto_patrimonio.merge(imposto_iss, left_index=True, right_index=True, how="outer", suffixes=('', '_y'))
    df_mesclada.drop(df_mesclada.filter(regex='_y$').columns, axis=1, inplace=True)
    df_completa = df_mesclada.merge(imposto_rf, left_index=True, right_index=True, how="outer", suffixes=('', '_y'))
    df_completa.drop(df_completa.filter(regex='_y$').columns, axis=1, inplace=True)
    
    if df_final.empty:
        df_final = df_completa
    else: 
        df_final = pd.concat([df_final, df_completa], axis=0)
    

---

Ordena o DataFrame final pelo índice.

Cria uma cópia do DataFrame final para testes.

Obtém uma lista de índices onde há valores nulos em qualquer coluna.

Inicializa um dicionário para armazenar os códigos de IBGE e os anos correspondentes aos valores nulos.

Para cada índice na lista de índices:

- Se o código IBGE não estiver no dicionário, adiciona o código com uma lista contendo o ano correspondente.
- Se o código IBGE já estiver no dicionário, adiciona o ano correspondente à lista existente.

---

In [ ]:
df_final.sort_index(inplace=True)
testes = df_final
indexes = list(testes.isnull().any(axis=1).index)
codigos = {}
for index in indexes:
    if index[0] not in codigos:
        codigos.update({index[0]:[index[1]]})
    else:
        codigos[index[0]].append(index[1])

---

Inicializa uma lista para armazenar códigos com erros.

Cria uma série booleana que indica quais linhas têm valores nulos.

Para cada código de IBGE no dicionário de códigos:

- Se o código não tiver 5 anos associados, adiciona o código à lista de erros e continua para o próximo código.
- Para cada ano associado ao código:
  - Se houver valores nulos na linha correspondente ao código e ano, adiciona o código à lista de erros e interrompe a verificação dos anos para esse código.

---

In [ ]:
erros = []
testes_null = testes.isnull().any(axis=1)
for codigo in codigos:
    if len(codigos[codigo]) != 5:
        erros.append(codigo)
        continue
    for ano in codigos[codigo]:
        if testes_null.loc[(codigo, ano)]:
            erros.append(codigo)
            break
            
        

---

Copia o DataFrame final para `df_bruta`.

Cria um DataFrame `df_limpa` excluindo as linhas que têm códigos de IBGE presentes na lista de erros.

Cria um DataFrame `df_erros` com os códigos de IBGE com erros e seus respectivos nomes de municípios.

Obtém uma lista ordenada de códigos de IBGE sem erros.

Cria um DataFrame `df_sucesso` com os códigos de IBGE sem erros e seus respectivos nomes de municípios.

---

In [ ]:
df_bruta = df_final

df_limpa = df_final[~df_final.index.get_level_values('Cod.IBGE').isin(erros)]

df_erros = pd.DataFrame({'Cod.IBGE': erros, "Municipios": [conversor[f] for f in erros]})

sucesso = sorted(list(set(df_limpa.index.get_level_values("Cod.IBGE"))))
df_sucesso = pd.DataFrame({"Cod.IBGE": sucesso, "Municipios": [conversor[f] for f in sucesso]})


---

Cria um objeto `ExcelWriter` para salvar os DataFrames em um arquivo Excel chamado "Output.xlsx".

Salva o DataFrame `df_bruta` na planilha "Dados Brutos", incluindo as colunas "População", "Município", "Impostos sobre o Patrimônio", "Impostos sobre Serviços" e "Impostos sobre Renda Retido na fonte", sem mesclar células.

Salva o DataFrame `df_limpa` na planilha "Dados Limpos", incluindo as colunas "População", "Município", "Impostos sobre o Patrimônio", "Impostos sobre Serviços" e "Impostos sobre Renda Retido na fonte", sem mesclar células.

Salva o DataFrame `df_erros` na planilha "Erros", sem incluir o índice.

Salva o DataFrame `df_sucesso` na planilha "Sucessos", sem incluir o índice.

Fecha o objeto `ExcelWriter` para salvar o arquivo.

---

In [ ]:
writer = pd.ExcelWriter("Output.xlsx",engine='openpyxl')   
df_bruta.to_excel(writer, sheet_name="Dados Brutos", columns=["População","Municipio", "Impostos sobre o Patrimônio", "Impostos sobre Serviços", "Impostos sobre Renda Retido na fonte"], merge_cells=False)
df_limpa.to_excel(writer, sheet_name="Dados Limpos", columns=["População","Municipio", "Impostos sobre o Patrimônio", "Impostos sobre Serviços", "Impostos sobre Renda Retido na fonte"], merge_cells=False)   
df_erros.to_excel(writer, sheet_name="Erros", index=False)
df_sucesso.to_excel(writer, sheet_name="Sucessos", index=False)
writer.close()